In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
df=lambda x:pd.DataFrame(x)

In [2]:
#键值对式序列化库
PICKLE_PATH='../softfits.pick'
import shelve
def set_pickle(k='temp',v=''):
    with shelve.open(PICKLE_PATH) as file:
        file[k]=v

def get_pickle(k):
    if k is None:
        return False
    with shelve.open(PICKLE_PATH) as file:
        try:
            v=file[k]
        except KeyError:
            return None
        else:
            return v

In [3]:
img=get_pickle('img1')

In [5]:
#考虑数据集的大小，只取中心的300x300的数据
aimg=img[150:450,150:450]

In [8]:
#依次
#地球=求地球位置半径(img)
#轮廓=求等离子层轮廓(img)
#太阳方向=求太阳方向(img,地球,轮廓)
地球=[309, 319,  59]#圆心，半径
太阳方向=-105

In [9]:
import math
center=(地球[0],地球[1])
def cart2pol(x, y):
    rho = np.sqrt((x-center[0])**2 + (y-center[1])**2)
    phi = np.arctan2((y-center[1]), (x-center[0]))
    return(rho, phi)

In [10]:
#指数衰减表
def dec(L):
    RR=地球[2]
    L=L-RR
    return np.exp(-L/RR) if L>0 else 0
dec(60)       

0.98319367626271825

In [11]:
targetModel=np.zeros((300,300,150))
for (x,y,z),v in np.ndenumerate(targetModel):
    p,s=cart2pol(x,y)
    L=math.sqrt(p**2+z**2)
    targetModel[x,y,z]=dec(L)  

In [19]:
def 线性归一(content):
    if content.max()==0:
        return content
    ss=content
    ss-=ss.min()
    ss=ss/ss.max()
    return ss

In [18]:
def 边界截断(seq,x,y):
    ss=seq.copy()
    p,s=cart2pol(x,y)
    L=地球[2]*4
    if L>p:
        h=int(math.sqrt(L**2-p**2))
        ss[h+1:]=0.0
    else:
        ss[:]=0.0
    return ss

In [21]:
ts=targetModel.shape
rs=np.array((ts[0],ts[1]))
Model=np.zeros((300,300,150))
cc=(((x,y),边界截断(targetModel[x,y,:],x,y)) for x,y in np.ndindex(ts[0],ts[1]))
cc=(((x,y),线性归一(v)) for (x,y),v in cc)
for (x,y),v in cc:
    Model[x,y,:]=v*aimg[x,y]


In [24]:
Model[200,150,:]

array([ 73.69660187,  73.69349633,  73.68418072,  73.66865809,
        73.64693351,  73.61901409,  73.58490893,  73.54462917,
        73.49818796,  73.44560041,  73.38688366,  73.32205679,
        73.25114087,  73.17415888,  73.09113577,  73.00209838,
        72.90707545,  72.8060976 ,  72.69919729,  72.58640883,
        72.46776833,  72.34331369,  72.21308457,  72.07712235,
        71.93547012,  71.78817266,  71.63527638,  71.47682932,
        71.31288109,  71.14348286,  70.96868732,  70.78854864,
        70.60312244,  70.41246575,  70.21663699,  70.0156959 ,
        69.80970355,  69.59872225,  69.38281555,  69.16204818,
        68.93648602,  68.70619605,  68.47124633,  68.23170592,
        67.9876449 ,  67.73913425,  67.4862459 ,  67.22905259,
        66.96762791,  66.70204621,  66.43238259,  66.15871281,
        65.88111331,  65.59966112,  65.31443384,  65.02550958,
        64.73296696,  64.43688501,  64.13734318,  63.83442125,
        63.52819935,  63.21875786,  62.9061774 ,  62.59